In [ ]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.ops import io_ops

In [ ]:
with tf.Session() as sess:
    #注意将模型移动/tmp目录下.
    saver = tf.train.import_meta_graph('/tmp/speech_commands_train/conv.ckpt-16200.meta')
    saver.restore(sess,tf.train.latest_checkpoint('/tmp/speech_commands_train'))
    var_list = {v.name: sess.run(v) for v in tf.global_variables()}
    for i in var_list.items():
        print(i[0], i[1].shape)

In [ ]:
saver1 = tf.train.Saver()
with tf.Session() as sess:   
    saver1.restore(sess,"/tmp/speech_commands_train/conv.ckpt-16200")
    var_list = {v.name: sess.run(v) for v in tf.global_variables()}
    for i in var_list.items():
        print(i[0], i[1].shape)
    

In [ ]:
sample_rate = 48000
desired_samples = sample_rate*0.5
window_size_samples = sample_rate*0.03
window_stride_samples = sample_rate*0.01
length_minus_window = (desired_samples - window_size_samples)
spectrogram_length = 1 + int(length_minus_window / window_stride_samples)
fingerprint_width = 40
fingerprint_size = spectrogram_length * fingerprint_width
label_count = 7
preprocess = 'mfcc'
average_window_width = -1

model_settings = {
      'desired_samples': desired_samples,
      'window_size_samples': window_size_samples,
      'window_stride_samples': window_stride_samples,
      'spectrogram_length': spectrogram_length,
      'fingerprint_width': fingerprint_width,
      'fingerprint_size': fingerprint_size,
      'label_count': label_count,
      'sample_rate': sample_rate,
      'preprocess': preprocess,
      'average_window_width': average_window_width,
  }
print(model_settings)

In [ ]:
#模型保存, placeholder放在fingerprint
g1 = tf.Graph()
import models
with g1.as_default():
    input_placeholder = tf.placeholder(
        tf.float32, [None, fingerprint_size], name='fingerprint_input')
    logits = models.create_model(
        input_placeholder,
        model_settings,
        'conv',
        is_training=False)
    var_list2 = {v.name: v for v in tf.global_variables()}
    for i in var_list2.items():
        print(i[0], i[1])
    var_op = {tf.assign(v[1],var_list[v[0]]) for v in var_list2.items()}
    
with tf.Session(graph=g1) as sess2:
    sess2.run(tf.global_variables_initializer())
    for i in var_op:
        sess2.run(i)
        converter = tf.lite.TFLiteConverter.from_session(sess2,
                                                         [input_placeholder],
                                                         [logits])
        tflite_model = converter.convert()
        open("fingerprint_input_model.tflite", "wb").write(tflite_model)

In [ ]:
#测试输出
filename = "../data/98K/audio_75490.688279.wav"
with tf.Session(graph=tf.Graph()) as sess:
    wav_filename_placeholder = tf.placeholder(tf.string, [])
    wav_loader = io_ops.read_file(wav_filename_placeholder)
    wav_decoder = contrib_audio.decode_wav(wav_loader, desired_channels=1)
    b = sess.run(wav_decoder,
                 feed_dict={wav_filename_placeholder: filename}).audio
    print(type(b))
    print(b.shape)
    str_b = ','.join(['%0.10f' %i[0] for i in b ])
    with open('audio.txt','wt') as f:
        f.write(str_b + '\n')


In [ ]:
#模型保存，placeholder放在audio_data
g2 = tf.Graph()
with g2.as_default():
    audio_data_placeholder = tf.placeholder(shape=[24000,1],dtype=tf.float32,name="autdio_ph")
    spectrogram = contrib_audio.audio_spectrogram(
        audio_data_placeholder,
        window_size=model_settings['window_size_samples'],
        stride=model_settings['window_stride_samples'],
        magnitude_squared=True)
    mfcc = contrib_audio.mfcc(
        spectrogram,
        model_settings['sample_rate'],
        dct_coefficient_count=model_settings['fingerprint_width'])
    shape_mfcc = tf.reshape(mfcc,shape=(1,model_settings['fingerprint_size']))
    logits = models.create_model(
        shape_mfcc,
        model_settings,
        'conv',
        is_training=False)
    var_list3 = {v.name: v for v in tf.global_variables()}
    for i in var_list3.items():
        print(i[0], i[1])
    var_op = {tf.assign(v[1],var_list[v[0]]) for v in var_list3.items()}
    
    
with tf.Session(graph=g2) as sess:
    sess.run(tf.global_variables_initializer())
    for i in var_op:
        sess.run(i)
    res = sess.run(logits, feed_dict={audio_data_placeholder: b})
    print(res)
    
    
with tf.Session(graph=g2) as sess:
    sess.run(tf.global_variables_initializer())
    for i in var_op:
        sess.run(i)
        
    from tensorflow.lite.python.convert import OpsSet
    converter = tf.lite.TFLiteConverter.from_session(sess,[audio_data_placeholder],[logits])
    converter.target_ops = set([OpsSet.TFLITE_BUILTINS,OpsSet.SELECT_TF_OPS])
    tflite_model = converter.convert()
    open("audio_input_model.tflite", "wb").write(tflite_model)